<a href="https://colab.research.google.com/github/MK316/F2024/blob/main/Multimedia/App_design1127.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application desgign samples: Verb tense/ Pluaral Nouns

In [ ]:
%%capture
!pip install gradio pandas

[verb sample file](https://raw.githubusercontent.com/MK316/241214/refs/heads/main/data/verb_sample.csv)



In [ ]:
#@markdown APP [1]: Verb tense
import gradio as gr
import pandas as pd

# Load the CSV file from the provided URL
csv_url = "https://raw.githubusercontent.com/MK316/241214/refs/heads/main/data/verb_sample.csv"
df = pd.read_csv(csv_url)

# Initialize score and trials
state = {"score": 0, "trials": 0}

# Function to display the selected verb details
def show_verb(index):
    if index < len(df):
        verb = df.iloc[index]['Verb']
        past = df.iloc[index]['Past']
        pp = df.iloc[index]['PP']
        return f"Verb: {verb}"
    else:
        return "Index out of range, please select a valid index."

# Function to check if the user's classification is correct
def check_answer(index, user_classification):
    if index < 0 or index >= len(df):
        return "Invalid index. Please try again.", f"Score: {state['score']}/{state['trials']}"

    verb = df.iloc[index]['Verb']
    past = df.iloc[index]['Past']
    pp = df.iloc[index]['PP']
    regularity = df.iloc[index]['Regularity']
    state["trials"] += 1
    if user_classification.lower() == regularity.lower():
        state["score"] += 1
        feedback = f"Correct! {verb} - {past} - {pp}"
    else:
        feedback = f"Incorrect. The correct answer is: {regularity}. Verb: {verb}, Past: {past}, PP: {pp}"
    return feedback, f"Score: {state['score']}/{state['trials']}"

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("### Verb Regularity Practice")
    gr.Markdown("Use the slider to select a verb index, then click 'Show the Verb' to see the verb and its forms. Answer the question and click 'Submit' to see your feedback.")

    # Slider to select verb index
    index = gr.Slider(0, len(df) - 1, step=1, label="Verb Index", value=0)
    show_button = gr.Button("Show the Verb")
    # Button to display verb
    verb_details = gr.Textbox(label="Verb Details", interactive=False)


    # Radio buttons for regularity question
    options = gr.Radio(["Regular", "Irregular"], label="Is the verb regular or irregular?")

    # Submit button and feedback
    submit_button = gr.Button("Submit")
    feedback = gr.Textbox(label="Feedback", interactive=False)
    score_display = gr.Textbox(label="Score/Trial", interactive=False)

    # Button click actions
    show_button.click(show_verb, inputs=index, outputs=verb_details)
    submit_button.click(check_answer, inputs=[index, options], outputs=[feedback, score_display])

# Launch the app
app.launch()


## APP [2] variation

In [ ]:
#@markdown APP2: Verb tense checking
import gradio as gr
import pandas as pd
import random

# Load the CSV file from the provided URL
csv_url = "https://raw.githubusercontent.com/MK316/241214/refs/heads/main/data/verb_sample.csv"
df = pd.read_csv(csv_url)

# Initialize state
state = {"remaining_verbs": df.copy(), "score": 0, "trials": 0, "current_index": -1}

# Function to select a random verb
def select_random_verb():
    if state["remaining_verbs"].empty:
        return "Completed"
    state["current_index"] = random.randint(0, len(state["remaining_verbs"]) - 1)
    selected_verb = state["remaining_verbs"].iloc[state["current_index"]]
    return f"Verb: {selected_verb['Verb']}"

# Function to check the user's classification
def check_answer(user_classification):
    if state["remaining_verbs"].empty:
        return "All verbs have been answered correctly. Great job!", f"Score: {state['score']}/{state['trials']}"

    index = state["current_index"]
    if index == -1:
        return "Please click 'Show the Verb' first.", f"Score: {state['score']}/{state['trials']}"

    verb_data = state["remaining_verbs"].iloc[index]
    verb = verb_data['Verb']
    past = verb_data['Past']
    pp = verb_data['PP']
    regularity = verb_data['Regularity']

    # Update trials
    state["trials"] += 1

    if user_classification.lower() == regularity.lower():
        state["score"] += 1
        feedback = f"Correct! {verb} - {past} - {pp}"
        # Remove the verb from the remaining list
        state["remaining_verbs"] = state["remaining_verbs"].drop(state["remaining_verbs"].index[index])
    else:
        feedback = f"Incorrect. The correct answer is: {regularity}. Verb: {verb}, Past: {past}, PP: {pp}"

    if state["remaining_verbs"].empty:
        return "Completed", f"Score: {state['score']}/{state['trials']}"

    return feedback, f"Score: {state['score']}/{state['trials']}"

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("### Verb Regularity Practice")
    gr.Markdown("A verb will be displayed randomly. Answer whether it's regular or irregular. Once answered correctly, the verb will be removed from the list. Complete all verbs to finish.")

    # Button to display a verb
    show_button = gr.Button("Show the Verb")
    verb_details = gr.Textbox(label="Verb Details", interactive=False)

    # Radio buttons for regularity question
    options = gr.Radio(["Regular", "Irregular"], label="Is the verb regular or irregular?")

    # Submit button and feedback
    submit_button = gr.Button("Submit")
    feedback = gr.Textbox(label="Feedback", interactive=False)
    score_display = gr.Textbox(label="Score/Trial", interactive=False)

    # Button click actions
    show_button.click(fn=select_random_verb, outputs=verb_details)
    submit_button.click(fn=check_answer, inputs=options, outputs=[feedback, score_display])

# Launch the app
app.launch()


# 2. Plural nouns (irregular forms)

[sample data](https://raw.githubusercontent.com/MK316/241214/refs/heads/main/data/verb_sample.csv)

In [ ]:
#@markdown APP [3] Plural noun 1
import gradio as gr
import pandas as pd
import random

# Sample irregular plural forms dataset
data = {
    "Singular": ["child", "foot", "tooth", "man", "woman", "mouse", "goose", "ox", "person", "cactus"],
    "Plural": ["children", "feet", "teeth", "men", "women", "mice", "geese", "oxen", "people", "cacti"]
}
df = pd.DataFrame(data)

# Initialize state
state = {"remaining_nouns": df.copy(), "score": 0, "trials": 0, "current_index": -1}

# Function to select a random noun
def select_random_noun():
    if state["remaining_nouns"].empty:
        return "Completed"
    state["current_index"] = random.randint(0, len(state["remaining_nouns"]) - 1)
    selected_noun = state["remaining_nouns"].iloc[state["current_index"]]
    return f"Singular: {selected_noun['Singular']}"

# Function to check the user's answer
def check_plural(user_plural):
    if state["remaining_nouns"].empty:
        return "All nouns have been answered correctly. Great job!", f"Score: {state['score']}/{state['trials']}"

    index = state["current_index"]
    if index == -1:
        return "Please click 'Show the Noun' first.", f"Score: {state['score']}/{state['trials']}"

    noun_data = state["remaining_nouns"].iloc[index]
    singular = noun_data['Singular']
    correct_plural = noun_data['Plural']

    # Update trials
    state["trials"] += 1

    if user_plural.lower() == correct_plural.lower():
        state["score"] += 1
        feedback = f"Correct! Singular: {singular}, Plural: {correct_plural}"
        # Remove the noun from the remaining list
        state["remaining_nouns"] = state["remaining_nouns"].drop(state["remaining_nouns"].index[index])
    else:
        feedback = f"Incorrect. The correct plural form is: {correct_plural}. Singular: {singular}"

    if state["remaining_nouns"].empty:
        return "Completed", f"Score: {state['score']}/{state['trials']}"

    return feedback, f"Score: {state['score']}/{state['trials']}"

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("### Irregular Plural Forms Practice")
    gr.Markdown("A singular noun will be displayed randomly. Type its plural form and submit your answer. Complete all nouns to finish.")

    # Button to display a noun
    show_button = gr.Button("Show the Noun")
    noun_details = gr.Textbox(label="Singular Noun", interactive=False)

    # Text input for plural form
    plural_input = gr.Textbox(label="Your Answer (Plural Form)")

    # Submit button and feedback
    submit_button = gr.Button("Submit")
    feedback = gr.Textbox(label="Feedback", interactive=False)
    score_display = gr.Textbox(label="Score/Trial", interactive=False)

    # Button click actions
    show_button.click(fn=select_random_noun, outputs=noun_details)
    submit_button.click(fn=check_plural, inputs=plural_input, outputs=[feedback, score_display])

# Launch the app
app.launch()


## Variation

In [ ]:
#@markdown APP [4] Plural noun practice
import gradio as gr
import pandas as pd
import random

# Sample irregular plural forms dataset
noun_url = "https://raw.githubusercontent.com/MK316/241214/refs/heads/main/data/verb_sample.csv"
df = pd.read_csv(noun_url)

# Initialize state
state = {"remaining_nouns": df.copy(), "score": 0, "trials": 0, "current_index": -1}

# Function to select a random noun
def select_random_noun():
    if state["remaining_nouns"].empty:
        return "Completed", "You've completed the practice!"
    state["current_index"] = random.randint(0, len(state["remaining_nouns"]) - 1)
    selected_noun = state["remaining_nouns"].iloc[state["current_index"]]
    return selected_noun["Singular"], "What's the plural form of this noun?"

# Function to check the user's answer
def check_plural(user_plural):
    if state["remaining_nouns"].empty:
        return "All nouns have been answered correctly. Great job!", f"Score: {state['score']}/{state['trials']}"

    index = state["current_index"]
    if index == -1:
        return "Please click 'Show the Noun' first.", f"Score: {state['score']}/{state['trials']}"

    noun_data = state["remaining_nouns"].iloc[index]
    singular = noun_data["Singular"]
    correct_plural = noun_data["Plural"]

    # Update trials
    state["trials"] += 1

    if user_plural.lower() == correct_plural.lower():
        state["score"] += 1
        feedback = f"Correct! '{correct_plural}' is the plural form of '{singular}'."
        # Remove the noun from the remaining list
        state["remaining_nouns"] = state["remaining_nouns"].drop(state["remaining_nouns"].index[index])
    else:
        feedback = f"Incorrect. The correct plural form is '{correct_plural}' for '{singular}'."

    if state["remaining_nouns"].empty:
        return "Completed", f"Score: {state['score']}/{state['trials']}"

    return feedback, f"Score: {state['score']}/{state['trials']}"

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("### Irregular Plural Forms Practice")
    gr.Markdown("Click 'Show the Noun' to display a singular noun. Type its plural form, submit your answer, and click 'Continue' to proceed to the next noun. Complete all nouns to finish.")

    # Button to display a noun
    show_button = gr.Button("Start your practice")
    noun_display = gr.Textbox(label="Singular Noun", value="Click 'Show the Noun' to start", interactive=False)
    prompt_display = gr.Textbox(label="Instruction", value="", interactive=False)

    # Text input for plural form
    plural_input = gr.Textbox(label="Your answer", placeholder="Type your answer here")

    # Submit button and feedback
    submit_button = gr.Button("See the Answer and Feedback")
    feedback_display = gr.Textbox(label="Feedback", interactive=False)
    score_display = gr.Textbox(label="Score/Trial", interactive=False)

    # Continue button for the next noun
    continue_button = gr.Button("Continue")

    # Button click actions
    show_button.click(fn=select_random_noun, outputs=[noun_display, prompt_display])
    submit_button.click(fn=check_plural, inputs=plural_input, outputs=[feedback_display, score_display])
    continue_button.click(fn=select_random_noun, outputs=[noun_display, prompt_display])

app.launch()


In [ ]:
#@markdown APP[5] Plural noun 3
import gradio as gr
import pandas as pd
import random

# Sample irregular plural forms dataset
data = {
    "Singular": ["child", "foot", "tooth", "man", "woman", "mouse", "goose", "ox", "person", "cactus"],
    "Plural": ["children", "feet", "teeth", "men", "women", "mice", "geese", "oxen", "people", "cacti"]
}
df = pd.DataFrame(data)

# Initialize state
state = {"remaining_nouns": df.copy(), "score": 0, "trials": 0, "current_index": -1}

# Function to select a random noun
def select_random_noun():
    if state["remaining_nouns"].empty:
        return "Completed"
    state["current_index"] = random.randint(0, len(state["remaining_nouns"]) - 1)
    selected_noun = state["remaining_nouns"].iloc[state["current_index"]]
    return f"What's the plural form of '{selected_noun['Singular']}'?"

# Function to check the user's answer
def check_plural(user_plural):
    if state["remaining_nouns"].empty:
        return "All nouns have been answered correctly. Great job!", f"Score: {state['score']}/{state['trials']}"

    index = state["current_index"]
    if index == -1:
        return "Please click 'Show the Noun' first.", f"Score: {state['score']}/{state['trials']}"

    noun_data = state["remaining_nouns"].iloc[index]
    singular = noun_data["Singular"]
    correct_plural = noun_data["Plural"]

    # Update trials
    state["trials"] += 1

    if user_plural.lower() == correct_plural.lower():
        state["score"] += 1
        feedback = f"Correct! '{correct_plural}' is the plural form of '{singular}'."
        # Remove the noun from the remaining list
        state["remaining_nouns"] = state["remaining_nouns"].drop(state["remaining_nouns"].index[index])
    else:
        feedback = f"Incorrect. The correct plural form is '{correct_plural}' for '{singular}'."

    if state["remaining_nouns"].empty:
        return "Completed", f"Score: {state['score']}/{state['trials']}"

    return feedback, f"Score: {state['score']}/{state['trials']}"

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("### Irregular Plural Forms Practice")
    gr.Markdown("Click 'Show the Noun' to display a singular noun. Type its plural form, submit your answer, and click 'Continue' to proceed to the next noun. Complete all nouns to finish.")

    # Button to display a noun
    show_button = gr.Button("Show the Noun")
    noun_display = gr.Textbox(label="Singular Noun", value="Click 'Show the Noun' to start", interactive=False)

    # Text input for plural form
    plural_input = gr.Textbox(label="Your answer", placeholder="Type your answer here")

    # Submit button and feedback
    submit_button = gr.Button("See the Answer and Feedback")
    feedback_display = gr.Textbox(label="Feedback", interactive=False)
    score_display = gr.Textbox(label="Score/Trial", interactive=False)

    # Continue button for the next noun
    continue_button = gr.Button("Continue")

    # Button click actions
    show_button.click(fn=select_random_noun, outputs=noun_display)
    submit_button.click(fn=check_plural, inputs=plural_input, outputs=[feedback_display, score_display])
    continue_button.click(fn=select_random_noun, outputs=noun_display)

app.launch()


[noun sample](https://raw.githubusercontent.com/MK316/241214/refs/heads/main/data/verb_sample.csv)

In [ ]:
#@markdown APP [6] Plural noun practice: short version
import gradio as gr
import pandas as pd
import random

# Sample irregular plural forms dataset
data = {
    "Singular": ["child", "foot", "tooth", "man", "woman", "mouse", "goose", "ox", "person", "cactus"],
    "Plural": ["children", "feet", "teeth", "men", "women", "mice", "geese", "oxen", "people", "cacti"]
}

# or using url
#data =
df = pd.DataFrame(data)

# Initialize state
state = {"remaining_nouns": df.copy(), "score": 0, "trials": 0, "current_index": -1}

# Function to select a random noun
def select_random_noun():
    if state["remaining_nouns"].empty:
        return "Completed"
    state["current_index"] = random.randint(0, len(state["remaining_nouns"]) - 1)
    selected_noun = state["remaining_nouns"].iloc[state["current_index"]]
    return f"What's the plural form of '{selected_noun['Singular']}'?"

# Function to check the user's answer
def check_plural(user_plural):
    if state["remaining_nouns"].empty:
        return "All nouns have been answered correctly. Great job! (Score: {state['score']}/{state['trials']})"

    index = state["current_index"]
    if index == -1:
        return "Please click 'Show the Noun' first. (Score: {state['score']}/{state['trials']})"

    noun_data = state["remaining_nouns"].iloc[index]
    singular = noun_data["Singular"]
    correct_plural = noun_data["Plural"]

    # Update trials
    state["trials"] += 1

    if user_plural.lower() == correct_plural.lower():
        state["score"] += 1
        feedback = f"Correct! '{correct_plural}' is the plural form of '{singular}'."
        # Remove the noun from the remaining list
        state["remaining_nouns"] = state["remaining_nouns"].drop(state["remaining_nouns"].index[index])
    else:
        feedback = f"Incorrect. The correct plural form is '{correct_plural}' for '{singular}'."

    if state["remaining_nouns"].empty:
        return f"All nouns have been answered correctly. Great job! (Score: {state['score']}/{state['trials']})"

    return f"{feedback} (Score: {state['score']}/{state['trials']})"

# Function to reset the input field and load the next noun
def reset_input_and_select_next():
    if state["remaining_nouns"].empty:
        return "Completed", ""
    next_noun = select_random_noun()
    return next_noun, ""  # Clear the input field

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("### Irregular Plural Forms Practice")
    gr.Markdown("Click 'Show the Noun' to display a singular noun. Type its plural form, submit your answer, and click 'Continue' to proceed to the next noun. Complete all nouns to finish.")

    # Button to display a noun
    show_button = gr.Button("Show the Noun")
    noun_display = gr.Textbox(label="Singular Noun", value="Click 'Show the Noun' to start", interactive=False)

    # Text input for plural form
    plural_input = gr.Textbox(label="Your answer", placeholder="Type your answer here")

    # Submit button and combined feedback + score display
    submit_button = gr.Button("See the Answer and Feedback")
    feedback_display = gr.Textbox(label="Feedback and Score", interactive=False)

    # Continue button for the next noun
    continue_button = gr.Button("Continue")

    # Button click actions
    show_button.click(fn=select_random_noun, outputs=noun_display)
    submit_button.click(fn=check_plural, inputs=plural_input, outputs=feedback_display)
    continue_button.click(
        fn=reset_input_and_select_next,
        outputs=[noun_display, plural_input]  # Clear the input field and show the next noun
    )

app.launch()


# Add your thoughts to further develop the applications

## Using sounds

In [ ]:
%%capture
!pip install gtts

In [ ]:
#@markdown APP [7] Verb tense practice with sounds
import pandas as pd
import gradio as gr
from gtts import gTTS
import tempfile
import os

# Step 1: Read the CSV file
csv_url = "https://raw.githubusercontent.com/MK316/241214/refs/heads/main/data/verb_sample.csv"
df = pd.read_csv(csv_url)

# Initialize state
state = {"index": 0, "completed": False}

# Function to generate text and audio for the current verb
def generate_practice():
    if state["completed"]:
        return "You've completed the practice.", None

    # Get the current verb data
    current_verb = df.iloc[state["index"]]
    verb_text = (
        f"Repeat after me: {current_verb['Verb']}, {current_verb['Past']}, {current_verb['PP']}."
    )

    # Generate audio using gTTS
    tts = gTTS(verb_text)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    tts.save(temp_file.name)

    return verb_text, temp_file.name

# Function to move to the next verb
def next_verb():
    if state["index"] < len(df) - 1:
        state["index"] += 1
    else:
        state["completed"] = True

    return generate_practice()

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("### Verb Practice")
    gr.Markdown("Click **Start** to begin practicing. Repeat after the audio and click **Next** to proceed to the next verb.")

    # Start button
    start_button = gr.Button("Start")
    text_display = gr.Textbox(label="Practice Text", interactive=False)
    audio_output = gr.Audio(label="Audio", interactive=False)

    # Next button
    next_button = gr.Button("Next")

    # Button actions
    start_button.click(fn=generate_practice, outputs=[text_display, audio_output])
    next_button.click(fn=next_verb, outputs=[text_display, audio_output])

# Launch the app
app.launch()


Plural noun application with sounds

[sample data](https://raw.githubusercontent.com/MK316/241214/refs/heads/main/data/verb_sample.csv)

In [ ]:
#@markdown APP [8] Plural form practice with sounds
import pandas as pd
import gradio as gr
from gtts import gTTS
import tempfile
import os

# Step 1: Read the CSV file
csv_url = "https://raw.githubusercontent.com/MK316/241214/refs/heads/main/data/plural_sample.csv"
df = pd.read_csv(csv_url)

# Initialize state
state = {"index": 0, "completed": False}

# Function to generate text and audio for the current noun
def generate_practice():
    if state["completed"]:
        return "You've completed the practice.", None

    # Get the current noun data
    current_noun = df.iloc[state["index"]]
    noun_text = (
        f"Repeat after me: the plural form of {current_noun['Singular']} is {current_noun['Plural']}."
    )

    # Generate audio using gTTS
    tts = gTTS(noun_text)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    tts.save(temp_file.name)

    return noun_text, temp_file.name

# Function to move to the next noun
def next_noun():
    if state["index"] < len(df) - 1:
        state["index"] += 1
    else:
        state["completed"] = True

    return generate_practice()

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("### Noun Practice")
    gr.Markdown("Click **Start** to begin practicing. Repeat after the audio and click **Next** to proceed to the next noun.")

    # Start button
    start_button = gr.Button("Start")
    text_display = gr.Textbox(label="Practice Text", interactive=False)
    audio_output = gr.Audio(label="Audio", interactive=False)

    # Next button
    next_button = gr.Button("Next")

    # Button actions
    start_button.click(fn=generate_practice, outputs=[text_display, audio_output])
    next_button.click(fn=next_noun, outputs=[text_display, audio_output])

# Launch the app
app.launch()
